In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv',
                   usecols=[2, 3,   7, 8, 9],
                   dtype={
                          'user_id': 'int32',
                          'content_id': 'int16',
                          
                          
                          'answered_correctly':'int8',
                          'prior_question_elapsed_time': 'float32',
                          'prior_question_had_explanation': 'boolean'}
                   )

In [ ]:
questions_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv',
                            usecols=[0,1, 3,4],
                            dtype={'question_id': 'int16',
                              'part': 'int8','bundle_id': 'int8','tags': 'str'}
                          )
tag = questions_df["tags"].str.split(" ", n = 10, expand = True) 
tag.columns = ['tags1','tags2','tags3','tags4','tags5','tags6']
tag['tagsum'] = 6 - tag.isnull().sum(axis=1)
tag['content_id'] = questions_df['question_id']

In [ ]:
user = pd.DataFrame(train,columns = ['user_id','content_id','answered_correctly','prior_question_elapsed_time'])
#位移一列位置
user.prior_question_elapsed_time = user.prior_question_elapsed_time.shift(-1)
user.prior_question_elapsed_time = user.prior_question_elapsed_time.fillna(0)
user_lectures = user[user.answered_correctly == -1]
user_question = user[user.answered_correctly != -1]
#增加空间

del user

In [ ]:
#bundle_id是指每道题的编号，有些大问题包含多个子问题，每个子问题编号未question_id,大题目编号是bundle_id

questions = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
lectures = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv')
questions = questions.rename(columns = {'question_id':'content_id'})
lectures = lectures.rename(columns = {'lecture_id':'content_id'})
questions = pd.DataFrame(questions,columns = ['content_id','part'])
user_lectures = pd.merge(user_lectures,lectures,on ='content_id',how = 'left')
user_question = pd.merge(user_question,questions,on ='content_id',how = 'left')

In [ ]:
user_question_part = pd.DataFrame(user_question,columns = ['user_id','part','answered_correctly'])
user_question_time = pd.DataFrame(user_question,columns = ['user_id','part','prior_question_elapsed_time'])
del user_question

In [ ]:
user_question_part_1 = user_question_part.groupby(['user_id','part'],as_index = False).sum()
user_question_part_2 = user_question_part.groupby(['user_id','part'],as_index = False).count()
del user_question_part

In [ ]:
user_question_part  = pd.merge(user_question_part_1,user_question_part_2,on =['user_id','part'],how = 'left')
user_question_part['correctly'] = user_question_part.answered_correctly_x/user_question_part.answered_correctly_y
user_question_part = user_question_part.rename(columns = {'answered_correctly_x':'answered_correctly','answered_correctly_y':'answered_count'})
user_question_part = user_question_part.drop(columns=['answered_correctly'])
user_question_time = user_question_time.groupby(['user_id','part'],as_index = False).sum()
user_question = pd.merge(user_question_part,user_question_time,on =['user_id','part'],how = 'left')
del user_question_part_1,user_question_part_2,user_question_part,user_question_time

In [ ]:
from __future__ import print_function

import math
import matplotlib.pyplot as plt

In [ ]:
u = user_question['answered_count'].mean()
std = user_question['answered_count'].std()
error = user_question[np.abs(user_question['answered_count'] - u) > 3*std]
user_question.loc[user_question.answered_count >= error.answered_count.min(),'answered_count'] = error.answered_count.min()
average = float(sum(user_question.answered_count))/len(user_question.answered_count)
total = 0

for value in user_question.answered_count:

    total += (value - average) ** 2
    
stddev = math.sqrt(total/len(user_question.answered_count))
user_question.answered_count = [(x-average)/stddev for x in user_question.answered_count]

In [ ]:
u = user_question['prior_question_elapsed_time'].mean()
std = user_question['prior_question_elapsed_time'].std()
error = user_question[np.abs(user_question['prior_question_elapsed_time'] - u) > 3*std]
user_question.loc[user_question.prior_question_elapsed_time >= error.prior_question_elapsed_time.min(),'prior_question_elapsed_time'] = error.prior_question_elapsed_time.min()
average = float(sum(user_question.prior_question_elapsed_time))/len(user_question.prior_question_elapsed_time)
total = 0

for value in user_question.prior_question_elapsed_time:

    total += (value - average) ** 2
    
stddev = math.sqrt(total/len(user_question.prior_question_elapsed_time))
user_question.prior_question_elapsed_time = [(x-average)/stddev for x in user_question.prior_question_elapsed_time]

In [ ]:
question_c = pd.DataFrame(train,columns = ['user_id','content_id','answered_correctly','prior_question_elapsed_time'])
question_c.prior_question_elapsed_time = question_c.prior_question_elapsed_time.shift(-1)
question_c.prior_question_elapsed_time = question_c.prior_question_elapsed_time.fillna(0)
question_c = question_c[question_c.answered_correctly != -1]
question_c = pd.merge(question_c,questions,on ='content_id',how = 'left')
question_c_part = pd.DataFrame(question_c,columns = ['content_id','part','answered_correctly'])
question_c_part_1 = question_c_part.groupby(['content_id','part'],as_index = False).sum()
question_c_part_2 = question_c_part.groupby(['content_id','part'],as_index = False).count()
del question_c_part

In [ ]:
question_c_part  = pd.merge(question_c_part_1,question_c_part_2,on =['content_id','part'],how = 'left')
question_c_part['correctly'] = question_c_part.answered_correctly_x/question_c_part.answered_correctly_y
question_c_part = question_c_part.rename(columns = {'answered_correctly_x':'answered_correctly','answered_correctly_y':'answered_count'})
question_c_part = question_c_part.drop(columns=['answered_correctly'])
question_c_part = pd.merge(question_c_part,tag,on = 'content_id',how = 'left')
question_c_part = question_c_part.drop(columns=['tags1','tags2','tags3','tags4','tags5','tags6'])
del question_c_part_1,question_c_part_2

In [ ]:
u = question_c_part['answered_count'].mean()
std = question_c_part['answered_count'].std()
error = question_c_part[np.abs(question_c_part['answered_count'] - u) > 3*std]
question_c_part.loc[question_c_part.answered_count >= error.answered_count.min(),'answered_count'] = error.answered_count.min()
average = float(sum(question_c_part.answered_count))/len(question_c_part.answered_count)
total = 0

for value in question_c_part.answered_count:

    total += (value - average) ** 2
    
stddev = math.sqrt(total/len(question_c_part.answered_count))
question_c_part.answered_count = [(x-average)/stddev for x in question_c_part.answered_count]

In [ ]:
u = question_c_part['tagsum'].mean()
std = question_c_part['tagsum'].std()
error = question_c_part[np.abs(question_c_part['tagsum'] - u) > 3*std]
question_c_part.loc[question_c_part.tagsum >= error.tagsum.min(),'tagsum'] = error.tagsum.min()
average = float(sum(question_c_part.tagsum))/len(question_c_part.tagsum)
total = 0

for value in question_c_part.tagsum:

    total += (value - average) ** 2
    
stddev = math.sqrt(total/len(question_c_part.tagsum))
question_c_part.tagsum = [(x-average)/stddev for x in question_c_part.tagsum]

In [ ]:
user_lectures_tag = pd.DataFrame(user_lectures,columns = ['user_id','part','tag'])
user_lectures_tag['temp'] = 1
user_lectures_tag = user_lectures_tag.groupby(['user_id','part','tag'],as_index = False).count()
user_lectures_tag = pd.DataFrame(user_lectures_tag,columns = ['user_id','part','tag'])
user_lectures_tag = user_lectures_tag.groupby(['user_id','part'],as_index = False).count()

In [ ]:
user_lectures_temp= pd.DataFrame(user_lectures,columns = ['user_id','part','type_of'])
user_lectures_intention = user_lectures_temp[user_lectures_temp.type_of == 'intention']
user_lectures_concept = user_lectures_temp[user_lectures_temp.type_of == 'concept']
user_lectures_solving = user_lectures_temp[user_lectures_temp.type_of == 'solving question']
user_lectures_intention = user_lectures_intention.groupby(['user_id','part'],as_index = False).count()
user_lectures_concept = user_lectures_concept.groupby(['user_id','part'],as_index = False).count()
user_lectures_solving = user_lectures_solving.groupby(['user_id','part'],as_index = False).count()
user_lectures_intention = user_lectures_intention.rename(columns = {'type_of':'intention'})
user_lectures_concept = user_lectures_concept.rename(columns = {'type_of':'concept'})
user_lectures_solving = user_lectures_solving.rename(columns = {'type_of':'solving'})



In [ ]:
user_lectures_lt = pd.DataFrame(user_lectures,columns = ['user_id','part','prior_question_elapsed_time'])
user_lectures_lt = user_lectures_lt.groupby(['user_id','part'],as_index = False).sum()

In [ ]:
u = user_lectures_intention['intention'].mean()
std = user_lectures_intention['intention'].std()
error = user_lectures_intention[np.abs(user_lectures_intention['intention'] - u) > 3*std]
user_lectures_intention.loc[user_lectures_intention.intention >= error.intention.min(),'intention'] = error.intention.min()
average = float(sum(user_lectures_intention.intention))/len(user_lectures_intention.intention)
total = 0

for value in user_lectures_intention.intention:

    total += (value - average) ** 2
    
stddev = math.sqrt(total/len(user_lectures_intention.intention))
user_lectures_intention.intention = [(x-average)/stddev for x in user_lectures_intention.intention]

In [ ]:
u = user_lectures_concept['concept'].mean()
std = user_lectures_concept['concept'].std()
error = user_lectures_concept[np.abs(user_lectures_concept['concept'] - u) > 3*std]
user_lectures_concept.loc[user_lectures_concept.concept >= error.concept.min(),'concept'] = error.concept.min()
average = float(sum(user_lectures_concept.concept))/len(user_lectures_concept.concept)
total = 0

for value in user_lectures_concept.concept:

    total += (value - average) ** 2
    
stddev = math.sqrt(total/len(user_lectures_concept.concept))
user_lectures_concept.concept = [(x-average)/stddev for x in user_lectures_concept.concept]

In [ ]:
u = user_lectures_solving['solving'].mean()
std = user_lectures_solving['solving'].std()
error = user_lectures_solving[np.abs(user_lectures_solving['solving'] - u) > 3*std]
user_lectures_solving.loc[user_lectures_solving.solving >= error.solving.min(),'solving'] = error.solving.min()
average = float(sum(user_lectures_solving.solving))/len(user_lectures_solving.solving)
total = 0

for value in user_lectures_solving.solving:

    total += (value - average) ** 2
    
stddev = math.sqrt(total/len(user_lectures_solving.solving))
user_lectures_solving.solving = [(x-average)/stddev for x in user_lectures_solving.solving]

In [ ]:
u = user_lectures_tag['tag'].mean()
std = user_lectures_tag['tag'].std()
error = user_lectures_tag[np.abs(user_lectures_tag['tag'] - u) > 3*std]
user_lectures_tag.loc[user_lectures_tag.tag >= error.tag.min(),'tag'] = error.tag.min()
average = float(sum(user_lectures_tag.tag))/len(user_lectures_tag.tag)
total = 0

for value in user_lectures_tag.tag:

    total += (value - average) ** 2
    
stddev = math.sqrt(total/len(user_lectures_tag.tag))
user_lectures_tag.tag = [(x-average)/stddev for x in user_lectures_tag.tag]

In [ ]:
u = user_lectures_lt['prior_question_elapsed_time'].mean()
std = user_lectures_lt['prior_question_elapsed_time'].std()
error = user_lectures_lt[np.abs(user_lectures_lt['prior_question_elapsed_time'] - u) > 3*std]
user_lectures_lt.loc[user_lectures_lt.prior_question_elapsed_time >= error.prior_question_elapsed_time.min(),'prior_question_elapsed_time'] = error.prior_question_elapsed_time.min()
average = float(sum(user_lectures_lt.prior_question_elapsed_time))/len(user_lectures_lt.prior_question_elapsed_time)
total = 0

for value in user_lectures_lt.prior_question_elapsed_time:

    total += (value - average) ** 2
    
stddev = math.sqrt(total/len(user_lectures_lt.prior_question_elapsed_time))
user_lectures_lt.prior_question_elapsed_time = [(x-average)/stddev for x in user_lectures_lt.prior_question_elapsed_time]

In [ ]:
user_question = pd.merge(user_question,user_lectures_intention,on =['user_id','part'],how = 'left')
user_question = pd.merge(user_question,user_lectures_concept,on =['user_id','part'],how = 'left')
user_question = pd.merge(user_question,user_lectures_solving,on =['user_id','part'],how = 'left')
user_question = pd.merge(user_question,user_lectures_tag,on =['user_id','part'],how = 'left')
user_question = pd.merge(user_question,user_lectures_lt,on =['user_id','part'],how = 'left')
user_question = user_question.rename(columns = {'prior_question_elapsed_time_x':'question_time','prior_question_elapsed_time_y':'lectrue_time'})

user_question = user_question.fillna(0)


In [ ]:
import matplotlib.pyplot as plt #Data Visualization 
import seaborn as sns  #Python library for Vidualization
from sklearn.cluster import KMeans


In [ ]:
user_question_part1 = user_question[user_question.part == 1]
user_question_part2 = user_question[user_question.part == 2]
user_question_part3 = user_question[user_question.part == 3]
user_question_part4 = user_question[user_question.part == 4]
user_question_part5 = user_question[user_question.part == 5]
user_question_part6 = user_question[user_question.part == 6]
user_question_part7 = user_question[user_question.part == 7]

In [ ]:
user_question_part1_m = pd.DataFrame(user_question_part1,columns = ['answered_count','correctly','question_time','intention','concept','solving','tag','lectrue_time'])
user_question_part2_m = pd.DataFrame(user_question_part2,columns = ['answered_count','correctly','question_time','intention','concept','solving','tag','lectrue_time'])
user_question_part3_m = pd.DataFrame(user_question_part3,columns = ['answered_count','correctly','question_time','intention','concept','solving','tag','lectrue_time'])
user_question_part4_m = pd.DataFrame(user_question_part4,columns = ['answered_count','correctly','question_time','intention','concept','solving','tag','lectrue_time'])
user_question_part5_m = pd.DataFrame(user_question_part5,columns = ['answered_count','correctly','question_time','intention','concept','solving','tag','lectrue_time'])
user_question_part6_m = pd.DataFrame(user_question_part6,columns = ['answered_count','correctly','question_time','intention','concept','solving','tag','lectrue_time'])
user_question_part7_m = pd.DataFrame(user_question_part7,columns = ['answered_count','correctly','question_time','intention','concept','solving','tag','lectrue_time'])

In [ ]:
kmeansmodel = KMeans(n_clusters= 5,max_iter = 100000,random_state = 0)


In [ ]:
y_kmeans_part1= kmeansmodel.fit_predict(user_question_part1_m)
y_kmeans_part2= kmeansmodel.fit_predict(user_question_part2_m)
y_kmeans_part3= kmeansmodel.fit_predict(user_question_part3_m)
y_kmeans_part4= kmeansmodel.fit_predict(user_question_part4_m)
y_kmeans_part5= kmeansmodel.fit_predict(user_question_part5_m)
y_kmeans_part6= kmeansmodel.fit_predict(user_question_part6_m)
y_kmeans_part7= kmeansmodel.fit_predict(user_question_part7_m)


In [ ]:
user_question_part1['type'] = y_kmeans_part1
user_question_part2['type'] = y_kmeans_part2
user_question_part3['type'] = y_kmeans_part3
user_question_part4['type'] = y_kmeans_part4
user_question_part5['type'] = y_kmeans_part5
user_question_part6['type'] = y_kmeans_part6
user_question_part7['type'] = y_kmeans_part7


In [ ]:
user_tz = pd.concat([user_question_part1,user_question_part2,user_question_part3,user_question_part4,user_question_part5,user_question_part6,user_question_part7])

In [ ]:
del user_question,user_question_part1_m,user_question_part2_m,user_question_part3_m,user_question_part4_m,user_question_part5_m
del user_question_part6_m,user_question_part7_m,user_question_part1,user_question_part2,user_question_part3,user_question_part4
del user_question_part5,user_question_part6,user_question_part7

In [ ]:
user_tz =pd.DataFrame(user_tz,columns = ['user_id','part','type'])

In [ ]:
question_c_part_part1 = question_c_part[question_c_part.part == 1]
question_c_part_part2 = question_c_part[question_c_part.part == 2]
question_c_part_part3 = question_c_part[question_c_part.part == 3]
question_c_part_part4 = question_c_part[question_c_part.part == 4]
question_c_part_part5 = question_c_part[question_c_part.part == 5]
question_c_part_part6 = question_c_part[question_c_part.part == 6]
question_c_part_part7 = question_c_part[question_c_part.part == 7]

In [ ]:
question_c_part_part1 = pd.DataFrame(question_c_part_part1,columns = ['answered_count','correctly','tagsum'])
question_c_part_part2 = pd.DataFrame(question_c_part_part2,columns = ['answered_count','correctly','tagsum'])
question_c_part_part3 = pd.DataFrame(question_c_part_part3,columns = ['answered_count','correctly','tagsum'])
question_c_part_part4 = pd.DataFrame(question_c_part_part4,columns = ['answered_count','correctly','tagsum'])
question_c_part_part5 = pd.DataFrame(question_c_part_part5,columns = ['answered_count','correctly','tagsum'])
question_c_part_part6 = pd.DataFrame(question_c_part_part6,columns = ['answered_count','correctly','tagsum'])
question_c_part_part7 = pd.DataFrame(question_c_part_part7,columns = ['answered_count','correctly','tagsum'])

In [ ]:
y_kmeans_part1= kmeansmodel.fit_predict(question_c_part_part1)
y_kmeans_part2= kmeansmodel.fit_predict(question_c_part_part2)
y_kmeans_part3= kmeansmodel.fit_predict(question_c_part_part3)
y_kmeans_part4= kmeansmodel.fit_predict(question_c_part_part4)
y_kmeans_part5= kmeansmodel.fit_predict(question_c_part_part5)
y_kmeans_part6= kmeansmodel.fit_predict(question_c_part_part6)
y_kmeans_part7= kmeansmodel.fit_predict(question_c_part_part7)

In [ ]:
question_c_part_part1['type'] = y_kmeans_part1
question_c_part_part2['type'] = y_kmeans_part2
question_c_part_part3['type'] = y_kmeans_part3
question_c_part_part4['type'] = y_kmeans_part4
question_c_part_part5['type'] = y_kmeans_part5
question_c_part_part6['type'] = y_kmeans_part6
question_c_part_part7['type'] = y_kmeans_part7

In [ ]:
question_tz = pd.concat([question_c_part_part1,question_c_part_part2,question_c_part_part3,question_c_part_part4,question_c_part_part5,question_c_part_part6,question_c_part_part7])

In [ ]:
question_tz['content_id'] = question_c_part.content_id
question_tz['part'] = question_c_part.part
question_tz =pd.DataFrame(question_tz,columns = ['content_id','part','type'])

In [ ]:
del question_c_part,question_c_part_part1,question_c_part_part2 ,question_c_part_part3,question_c_part_part4
del question_c_part_part5,question_c_part_part6,question_c_part_part7
del y_kmeans_part1,y_kmeans_part2,y_kmeans_part3,y_kmeans_part4,y_kmeans_part5,y_kmeans_part6,y_kmeans_part7
del questions_df,tag

In [ ]:
train.prior_question_elapsed_time = train.prior_question_elapsed_time.shift(-1)
train.prior_question_elapsed_time = train.prior_question_elapsed_time.fillna(0)

In [ ]:

train = pd.merge(train,question_tz,on = ['content_id'],how = 'left')


In [ ]:
train = pd.merge(train,user_tz,on = ['user_id','part'],how = 'left')


In [ ]:
y = train['answered_correctly']
train = train.drop(['answered_correctly'], axis=1)

In [ ]:
import lightgbm as lgb
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train,y,test_size = 0.4,random_state = 0)

In [ ]:
del train,y

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()

X_train.prior_question_had_explanation.fillna(False, inplace = True)
X_test.prior_question_had_explanation.fillna(False, inplace = True)
X_train["prior_question_had_explanation_enc"] = lb_make.fit_transform(X_train["prior_question_had_explanation"])
X_test["prior_question_had_explanation_enc"] = lb_make.fit_transform(X_test["prior_question_had_explanation"])

X_train = X_train.drop(['prior_question_had_explanation'], axis=1)
X_test = X_test.drop(['prior_question_had_explanation'], axis=1)

                       

In [ ]:
X_train.to_csv("X_train.csv", index=0)
X_test.to_csv("X_test.csv", index=0)
y_train.to_csv("y_train.csv", index=0)
y_test.to_csv("y_test.csv", index=0)
question_tz.to_csv("question_tz.csv", index=0)
user_tz.to_csv("user_tz.csv", index=0)